# CA6000 Assignment - K-Pop Girl Group Lyrics Generation Prediction

**Student**: ZHANG XINYING  
**Date**: 27 December 2025

This notebook contains the complete code for the assignment: data import, intentional error introduction, cleaning, statistics, preprocessing, MLP model training and evaluation.

## 1. Data Import and Initial Inspection

In [ ]:
import pandas as pd

df = pd.read_csv('girlgroup_songs.csv', encoding='utf-8-sig')
print("Dataset Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.info()
df.head()

In [ ]:
print("Missing Values:\n", df.isnull().sum())
print("Duplicate Rows:", df.duplicated().sum())

## 2. Intentionally Introduce Errors & Cleaning

In [ ]:
import numpy as np
np.random.seed(42)

df_error = df.copy()

# Introduce errors
lyrics_nan_idx = np.random.choice(df_error.index, size=int(len(df_error)*0.05), replace=False)
df_error.loc[lyrics_nan_idx, 'lyrics'] = np.nan

artist_nan_idx = np.random.choice(df_error.index, size=int(len(df_error)*0.03), replace=False)
df_error.loc[artist_nan_idx, 'artist'] = np.nan

duplicates = df_error.sample(n=50, random_state=42)
df_error = pd.concat([df_error, duplicates], ignore_index=True)

invalid_year_idx = df_error.sample(n=10, random_state=42).index
df_error.loc[invalid_year_idx, 'year'] = 'invalid'

outlier_idx = df_error.sample(n=15, random_state=42).index
df_error.loc[outlier_idx[:10], 'lyrics_length'] = 99999
df_error.loc[outlier_idx[10:], 'lyrics_length'] = -500

print("After introducing errors - Shape:", df_error.shape)

In [ ]:
# Cleaning steps
print("Initial shape with errors:", df_error.shape)

df_error = df_error.drop_duplicates()
print("After drop_duplicates:", df_error.shape)

df_error = df_error.dropna(subset=['lyrics', 'artist'])
print("After dropna critical:", df_error.shape)

df_error['year'] = pd.to_numeric(df_error['year'], errors='coerce')
df_error = df_error.dropna(subset=['year'])
df_error['year'] = df_error['year'].astype(int)
print("After year fix:", df_error.shape)

df_error = df_error[(df_error['lyrics_length'] >= 50) & (df_error['lyrics_length'] <= 5000)]
print("Final clean shape:", df_error.shape)

## 3. Statistical Summary

In [ ]:
stats = df_error[['year', 'lyrics_length', 'chart_rank']].describe()
stats.loc['variance'] = df_error[['year', 'lyrics_length', 'chart_rank']].var()
stats.round(2)

In [ ]:
def assign_generation(debut_year):
    if 1996 <= debut_year <= 2002: return 'Gen 1'
    elif 2003 <= debut_year <= 2009: return 'Gen 2'
    elif 2010 <= debut_year <= 2013: return 'Gen 3'
    elif 2014 <= debut_year <= 2017: return 'Gen 4'
    else: return 'Gen 5'

df_error['generation'] = df_error['debut_year'].apply(assign_generation)

gen_dist = df_error['generation'].value_counts().sort_index()
gen_pct = (gen_dist / len(df_error) * 100).round(1)
pd.concat([gen_dist, gen_pct], axis=1, keys=['Count', 'Percentage'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.figure(figsize=(10,5))
sns.countplot(data=df_error, x='generation', order=['Gen 1','Gen 2','Gen 3','Gen 4','Gen 5'])
plt.title('Generation Distribution')
plt.show()

plt.figure(figsize=(8,5))
df_error['lyrics_length'].hist(bins=30)
plt.title('Lyrics Length Distribution')
plt.xlabel('Length (characters)')
plt.ylabel('Frequency')
plt.show()

## 4. Preprocessing & Model Training

In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Text cleaning
df_error['clean_lyrics'] = df_error['lyrics'].apply(lambda x: re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', str(x).lower()))

# TF-IDF
vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
X = vectorizer.fit_transform(df_error['clean_lyrics'])

# Labels
le = LabelEncoder()
y = le.fit_transform(df_error['generation'])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

In [ ]:
model = MLPClassifier(
    hidden_layer_sizes=(256, 128, 64),
    activation='relu',
    solver='adam',
    early_stopping=True,
    validation_fraction=0.1,
    max_iter=200,
    random_state=42
)

model.fit(X_train, y_train)
print("Iterations completed:", model.n_iter_)
print("Best validation score:", round(model.best_validation_score_, 4) if hasattr(model, 'best_validation_score_') else "N/A")

In [ ]:
y_pred = model.predict(X_test)
print("Test Accuracy:", round(accuracy_score(y_test, y_pred), 4))
print(classification_report(y_test, y_pred, target_names=le.classes_))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()